## Optional - make restraints
This will make restraints and generate configs for them.
Restraints between molecules has to be done using pull code added to MDP file.
See here http://www.mdtutorials.com/gmx/umbrella/05_pull.html
Index file need to be generated
```
---Example of pull code below ---
; Pull code
pull                    = yes
pull_ncoords            = 1         ; only one reaction coordinate
pull_ngroups            = 2         ; two groups defining one reaction coordinate
pull_group1_name        = Chain_A
pull_group2_name        = Chain_B
pull_coord1_type        = umbrella  ; harmonic potential
pull_coord1_geometry    = distance  ; simple distance increase
pull_coord1_dim         = N N Y     ; pull along z
pull_coord1_groups      = 1 2       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord1_start       = yes       ; define initial COM distance > 0
pull_coord1_rate        = 0.01      ; 0.01 nm per ps = 10 nm per ns
pull_coord1_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
```

In [3]:
#Libraries and main variables initialized
import MDAnalysis as mda
from MDAnalysis.analysis import align
from MDAnalysis.analysis.rms import rmsd
import numpy as np
import urllib.request, json
import os
import re
import nglview as nv
from funcs import get_files_from_git
from funcs import view_nucl
from IPython.display import Markdown, display
import pandas as pd
#Reloading funcs if changed
from importlib import reload 
import funcs,config
reload(funcs)
reload(config)

%matplotlib inline

from IPython.display import Markdown, display
from config import folder,descr,pname,ssh_host
display(Markdown(descr))
print("Folder set to: ",folder)
print("Project name: ",pname)
print("SSH host set to:",ssh_host)





# MD simulations of a tetrasome, 61 DNA bp, histone tails maximally truncated
- AMBER14SB force field with CUFIX
- 150 mM NaCl
- box 2nm


Folder set to:  h3-h4_tm_2_60DNA
Project name:  h3-h4_tm_2_60DNA
SSH host set to: lomo2k


In [5]:
import MDAnalysis.analysis.hbonds
import MDAnalysis.analysis.distances


nucl=mda.Universe("GMX_system/sys_ref.pdb")


sel_end1_1="(segid I and resnum -30 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"
sel_end1_2="(segid J and resnum 30 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"

sel_end2_1="(segid I and resnum 30 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"
sel_end2_2="(segid J and resnum -30 and ((resname DC DT and name N1) or (resname DG DA and name N9)))"

N=nucl.select_atoms("(segid I and resnum -30) or (segid J and resnum 30)")


E1NI=nucl.select_atoms(sel_end1_1)
E1NJ=nucl.select_atoms(sel_end1_2)

d1=MDAnalysis.analysis.distances.dist(E1NI,E1NJ)[2][0]/10.
print(d1)

E2NI=nucl.select_atoms(sel_end2_1)
E2NJ=nucl.select_atoms(sel_end2_2)

d2=MDAnalysis.analysis.distances.dist(E2NI,E2NJ)[2][0]/10.
print(d2)

#N1=nucl.select_atoms(sel1+" or "+sel2)

#h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(nucl, sel1, sel2, distance=3.0, angle=120.0)
#h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(nucl, 'nucleic', 'nucleic', distance=3.0, angle=120.0)
#h.run()

w=nv.show_mdanalysis(N,gui=False)
#w.add_representation( "label", ".N9", color="grey" )
w

0.9147212650699436
0.9271493815437994


NGLWidget()

In [6]:
e1ni=E1NI.atoms[0].id
e1nj=E1NJ.atoms[0].id
e2ni=E2NI.atoms[0].id
e2nj=E2NJ.atoms[0].id



Let's generate distance restraints
http://manual.gromacs.org/current/manual-2018.4.pdf


In [8]:
%%bash -s "$d1" "$d2" "$e1ni" "$e1nj" "$e2ni" "$e2nj" 
cat > GMX_system/pull.mdp <<!
; Pull code
pull                    = yes
pull_ncoords            = 2         ; only one reaction coordinate
pull_ngroups            = 4         ; two groups defining one reaction coordinate
pull_group1_name        = a_$3
pull_group2_name        = a_$4
pull_group3_name        = a_$5
pull_group4_name        = a_$6
pull_coord1_type        = umbrella  ; harmonic potential
pull_coord1_geometry    = distance  ; simple distance increase
pull_coord1_dim         = Y Y Y     ; pull along z
pull_coord1_groups      = 1 2       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord1_start       = no       ; define initial COM distance > 0
pull_coord1_rate        = 0.00      ; 0.01 nm per ps = 10 nm per ns
pull_coord1_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
pull-coord1-init        = $1

pull_coord2_type        = umbrella  ; harmonic potential
pull_coord2_geometry    = distance  ; simple distance increase
pull_coord2_dim         = Y Y Y     ; pull along z
pull_coord2_groups      = 3 4       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord2_start       = no       ; define initial COM distance > 0
pull_coord2_rate        = 0.00      ; 0.01 nm per ps = 10 nm per ns
pull_coord2_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
pull-coord2-init        = $2
!

In [9]:
!cat GMX_system/pull.mdp


; Pull code
pull                    = yes
pull_ncoords            = 2         ; only one reaction coordinate
pull_ngroups            = 4         ; two groups defining one reaction coordinate
pull_group1_name        = a_11
pull_group2_name        = a_3851
pull_group3_name        = a_1915
pull_group4_name        = a_1947
pull_coord1_type        = umbrella  ; harmonic potential
pull_coord1_geometry    = distance  ; simple distance increase
pull_coord1_dim         = Y Y Y     ; pull along z
pull_coord1_groups      = 1 2       ; groups 1 (Chain A) and 2 (Chain B) define the reaction coordinate
pull_coord1_start       = no       ; define initial COM distance > 0
pull_coord1_rate        = 0.00      ; 0.01 nm per ps = 10 nm per ns
pull_coord1_k           = 1000      ; kJ mol^-1 nm^-2, 4 kT per A which is ok
pull-coord1-init        = 0.9147212650699436

pull_coord2_type        = umbrella  ; harmonic potential
pull_coord2_geometry    = distance  ; simple distance increase
pull_coord2_dim        

In [10]:
%%bash -s "$d1" "$d2" "$e1ni" "$e1nj" "$e2ni" "$e2nj" 
cd GMX_system

gmx make_ndx -f init_solv_ions.pdb -n index.ndx -o index.ndx <<!
a $3
a $4
a $5
a $6
q
!

Going to read 1 old index file(s)

  0 System              : 245092 atoms
  1 DNA                 :  3872 atoms
  2 NA                  :   303 atoms
  3 CL                  :   211 atoms
  4 Protein             :  5734 atoms
  5 Protein-H           :  2788 atoms
  6 C-alpha             :   344 atoms
  7 Backbone            :  1036 atoms
  8 MainChain           :  1388 atoms
  9 MainChain+Cb        :  1710 atoms
 10 MainChain+H         :  1726 atoms
 11 SideChain           :  4008 atoms
 12 SideChain-H         :  1400 atoms
 13 Prot-Masses         :  5734 atoms
 14 non-Protein         : 239358 atoms
 15 Water               : 234972 atoms
 16 SOL                 : 234972 atoms
 17 non-Water           : 10120 atoms
 18 Ion                 :   514 atoms
 19 NA                  :   303 atoms
 20 CL                  :   211 atoms
 21 Water_and_ions      : 235486 atoms
 22 !Water_and_ions     :  9606 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': 

gmx: Relink `/opt/miniconda3/envs/moldyn/bin/../lib/././libgfortran.so.4' with `/lib/x86_64-linux-gnu/librt.so.1' for IFUNC symbol `clock_gettime'
                     :-) GROMACS - gmx make_ndx, 2018.4 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar    Aldert van Buuren   Rudi van Drunen     Anton Feenstra  
  Gerrit Groenhof    Aleksei Iupinov   Christoph Junghans   Anca Hamuraru   
 Vincent Hindriksen Dimitrios Karkoulis    Peter Kasson        Jiri Kraus    
  Carsten Kutzner      Per Larsson      Justin A. Lemkul    Viveca Lindahl  
  Magnus Lundborg   Pieter Meulenhoff    Erik Marklund      Teemu Murtola   
    Szilard Pall       Sander Pronk      Roland Schulz     Alexey Shvetsov  
   Michael Shirts     Alfons Sijbers     Peter Tieleman    Teemu Virolainen 
 Christian Wennberg    Maarten Wolf   
                           and the project leaders:
        Mark Abraham, Berk Hes

In [33]:
%%bash
##Patch mdp files
cp GMX_system/pull.mdp MDProtocols/
cat MDProtocols/pull.mdp >> MDProtocols/1_minim.mdp
cat MDProtocols/pull.mdp >> MDProtocols/2_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/3_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/4_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/5_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/6_equil.mdp
cat MDProtocols/pull.mdp >> MDProtocols/7_prod.mdp
cat MDProtocols/pull.mdp >> MDProtocols/bench.mdp

In [12]:
!cat MDProtocols/pull.mdp >> MDProtocols/7_prod_10ps_out.mdp

